## Funcionamiento de las Transacciones de Amazon DynamoDB

Con Amazon DynamoDB Transactions, puede agrupar varias acciones y enviarlas como una sola operación TransactWriteItems o TransactGetItems de tipo "todo o nada". Estas acciones pueden dirigirse a elementos de diferentes tablas, pero no a diferentes cuentas o regiones de AWS, y no puede haber dos acciones dirigidas al mismo elemento. Las acciones se completan de forma atómica, de modo que todas ellas tienen éxito o todas ellas fallan.

#### API TransactWriteItems

`TransactWriteItems` es una operación de escritura síncrona e idempotente que agrupa hasta 100 acciones de escritura en una única operación de tipo “todo o nada”. Estas acciones pueden estar dirigidas a un máximo de 100 elementos diferenciados en una o varias tablas de DynamoDB en la misma cuenta de AWS y en la misma región. El tamaño total de los elementos de la transacción no puede superar 4 MB. Las acciones se realizan atómicamente, de tal forma que se llevan a cabo correctamente todas o ninguna de ellas.

* Una operación `TransactWriteItems` se diferencia de una operación `BatchWriteItem` en que todas las acciones que contiene deben completarse correctamente o, de lo contrario, no se realiza ningún cambio. Con una operación BatchWriteItem, es posible que solo algunas de las acciones del lote se realicen correctamente y otras, no.

* Las transacciones no se pueden realizar mediante índices.

No se pueden dirigir varias operaciones de la misma transacción al mismo elemento. Por ejemplo, no se puede realizar una acción ConditionCheck y también una acción Update en el mismo elemento de la misma transacción.

Puede agregar los tipos de acciones siguientes a una transacción:

* *Put*: Inicia una operación PutItem para crear un nuevo elemento o sustituir uno antiguo por otro nuevo, ya sea de forma condicional o sin especificar ninguna condición.

* *Update*: Inicia una operación UpdateItem para editar los atributos de un elemento existente o agregar un nuevo elemento a la tabla, si no existe ya. Esta acción se utiliza para agregar, eliminar o actualizar atributos a un elemento existente, de forma condicional o sin condiciones.

* *Delete*: Inicia una operación DeleteItem para eliminar un solo elemento de una tabla identificado por su clave principal.

* *ConditionCheck*: Comprueba la existencia de un elemento o la condición de atributos concretos del elemento.

#### Idempotencia
Opcionalmente, puede incluir un token de cliente al realizar una llamada a TransactWriteItems para asegurarse de que la solicitud sea idempotente. Hacer que las transacciones sean idempotentes ayuda a evitar errores de aplicaciones si se envía la misma operación varias veces debido a una interrupción de la conexión u otro problema de conectividad.

Si la llamada a TransactWriteItems original se realizó correctamente, las llamadas siguientes a TransactWriteItems con el mismo token de cliente también se devolverán correctamente sin hacer ningún cambio. Si se ha establecido el parámetro ReturnConsumedCapacity, la llamada TransactWriteItems inicial devolverá el número de unidades de capacidad de escritura consumidas al realizar los cambios. Las llamadas siguientes a TransactWriteItems con el mismo token de cliente devolverán el número de unidades de capacidad de lectura consumidas al leer el elemento.

Información importante sobre la idempotencia:

* Un token de cliente es válido durante 10 minutos desde que finaliza la solicitud que lo utiliza. Transcurridos esos 10 minutos, cualquier solicitud que use el mismo token de cliente se tratará como si fuera nueva. No debe reutilizar el mismo token de cliente para la misma solicitud si han pasado 10 minutos.

* Si repite una solicitud con el mismo token de cliente dentro del periodo de idempotencia de 10 minutos, pero cambia algún otro parámetro de la solicitud, DynamoDB devolverá una excepción IdempotentParameterMismatch.

#### Control de errores de escritura
Las transacciones de escritura no se realizarán correctamente en las siguientes circunstancias:

* Cuando no se cumple alguna de las condiciones de las expresiones de condición.

* Cuando se produce un error de validación de transacción porque hay más de una acción dirigida al mismo elemento en una sola operación TransactWriteItems.

* Cuando una solicitud TransactWriteItems está en conflicto con una operación TransactWriteItems en curso para uno o varios elementos de la solicitud TransactWriteItems. En este caso, la solicitud genera un error TransactionCanceledException.

* Cuando no hay suficiente capacidad aprovisionada para completar la transacción.

* Cuando el tamaño de un elemento es excesivo (más de 400 KB), un índice secundario local (LSI) se vuelve demasiado grande o se produce algún error de validación semejante a causa de los cambios realizados por la transacción.

* Cuando hay algún error de usuario, como un formato de datos no válido.

In [1]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
import base64
from spdynamodb import DynamoTable
from datetime import datetime
import json
import time
from decimal import Decimal

In [2]:
#dt = DynamoTable(profile_name='my-profile')
dt=DynamoTable()
try:
    dt.select_table('Bookks')
    print(dt)
except:
    dt.create_table(
        table_name='Bookks',
        partition_key='Author',
        partition_key_type='S',
        sort_key='Title',
        sort_key_type='S',
    )

- Table name: Bookks            
- Table arn: arn:aws:dynamodb:us-east-1:089715336747:table/Bookks            
- Table creation: 2023-05-27 19:49:51            
- [{'AttributeName': 'Author', 'KeyType': 'HASH'}, {'AttributeName': 'Title', 'KeyType': 'RANGE'}]            
- [{'AttributeName': 'Author', 'AttributeType': 'S'}, {'AttributeName': 'Category', 'AttributeType': 'S'}, {'AttributeName': 'Title', 'AttributeType': 'S'}]            
- Point-in-time recovery status: ENABLED  |  Delete protection: False


In [3]:
dt.load_json("partiqlbatch.json")

Data loaded successfully from partiqlbatch.json.


In [4]:
client = boto3.client('dynamodb', region_name='us-east-1')

Las transacciones en DynamoDB respetan el concepto de idempotencia. La idempotencia permite enviar la misma transacción más de una vez, pero DynamoDB solo la ejecutará una vez. Esto resulta especialmente útil cuando se utilizan API que no son idempotentes, como UpdateItem para aumentar o disminuir un campo numérico, por ejemplo. Cuando se ejecuta una transacción se especifica una cadena que representa el ClientRequestToken (también conocido como Idempotency Token). DynamoDB utiliza este token para identificar transacciones duplicadas y evitar que se ejecuten más de una vez. Si se envía una transacción con un ClientRequestToken que ya se ha utilizado, DynamoDB devuelve un error de transacción.

In [5]:
token = "TRANSACTION1"

In [21]:
try:
    response = client.transact_write_items(
        TransactItems=[
            {
                'Put': {
                    'TableName': 'Bookks',
                    'Item': {
                        'Author': {'S': 'Franz Kafka'},
                        'Title': {'S': 'The Trial'},
                        'Category': {'S': 'Fiction'},
                        'Formats': {
                            'M': {
                                'Hardcover': {'S': 'ISBN 978-3-16-148410-0'},
                                'Paperback': {'S': 'ISBN 978-3-16-148409-4'},
                                'Ebook': {'S': 'ISBN 978-3-16-148408-7'},
                            }
                        },
                        'Year': {'N': '1925'},
                        'Pages': {'N': '255'}
                    }
                }
            },
            {
                'Put': {
                    'TableName': 'Bookks',
                    'Item': {
                        'Author': {'S': 'Franz Kafka'},
                        'Title': {'S': 'The Castle'},
                        'Category': {'S': 'Fiction'},
                        'Formats': {
                            'M': {
                                'Hardcover': {'S': 'ISBN 978-3-16-148410-0'},
                                'Paperback': {'S': 'ISBN 978-3-16-148409-4'},
                                'Ebook': {'S': 'ISBN 978-3-16-148408-7'},
                            }
                        },
                        'Year': {'N': '1926'},
                        'Pages': {'N': '315'}
                    }
                }
            },
            {
                'Update': {
                    'TableName': 'Bookks',
                    'Key': {
                        'Author': {'S': 'William Shakespeare'},
                        'Title': {'S': 'The Tempest'}
                    },
                    'UpdateExpression': 'SET #c = :val',
                    'ExpressionAttributeNames': {'#c': 'Category'},
                    'ExpressionAttributeValues': {':val': {'S': 'Drama'}}
                }
            }
        ],
        ClientRequestToken=token,
        ReturnConsumedCapacity='TOTAL'
    )
except ClientError as e:
    print('Error:', e.response['Error']['Message'])
else:
    print('Books updated.')
    print("ConsumedCapacity: ", response['ConsumedCapacity'])

Books updated.
ConsumedCapacity:  [{'TableName': 'Bookks', 'CapacityUnits': 6.0, 'ReadCapacityUnits': 6.0}]


Si ejecuta de nuevo otro comando de transacción, con el mismo valor de cliente-request-token, puede verificar que las otras invocaciones de la transacción son esencialmente ignoradas y los Mensajes atribuidos permanecen iguales.

In [23]:
try:
    response = client.transact_write_items(
        TransactItems=[
            {
                'Put': {
                    'TableName': 'Bookks',
                    'Item': {
                        'Author': {'S': 'Franz Kafka'},
                        'Title': {'S': 'The Trial'},
                        'Category': {'S': 'Fiction'},
                        'Formats': {
                            'M': {
                                'Hardcover': {'S': 'ISBN 978-3-16-148410-0'}
                            }
                        },
                        'Year': {'N': '1525'},
                        'Pages': {'N': '12'}
                    }
                }
            },
            {
                'Update': {
                    'TableName': 'Bookks',
                    'Key': {
                        'Author': {'S': 'William Shakespeare'},
                        'Title': {'S': 'The Tempest'}
                    },
                    'UpdateExpression': 'SET #c = :val',
                    'ExpressionAttributeNames': {'#c': 'Category'},
                    'ExpressionAttributeValues': {':val': {'S': 'Terror'}}
                }
            }
        ],
        ClientRequestToken=token,
        ReturnConsumedCapacity='TOTAL'
    )
except ClientError as e:
    print('Error:', e.response['Error']['Message'])
else:
    print('Books updated.')
    print("ConsumedCapacity: ", response['ConsumedCapacity'])

Error: Specified idempotent token was used with different request parameters within the idempotency window


#### API TransactGetItems
TransactGetItems es una operación de lectura síncrona que agrupa hasta 100 acciones Get. Estas acciones pueden estar dirigidas a un máximo de 100 elementos diferenciados en una o varias tablas de DynamoDB en la misma cuenta y región de AWS. El tamaño total de los elementos de la transacción no puede superar 4 MB.

Las acciones Get se realizan atómicamente, de tal forma que se llevan a cabo correctamente todas o ninguna de ellas:

* *Get*: Inicia una operación GetItem para recuperar un conjunto de atributos para el elemento que tiene la clave principal especificada. Si no se encuentra ningún elemento que coincida, Get no devuelve ningún dato.

#### Control de errores de lectura
Las transacciones de lectura no se realizarán correctamente en las siguientes circunstancias:

* Cuando una solicitud TransactGetItems está en conflicto con una operación TransactWriteItems en curso para uno o varios elementos de la solicitud TransactGetItems. En este caso, la solicitud genera un error TransactionCanceledException.

* Cuando no hay suficiente capacidad aprovisionada para completar la transacción.

* Cuando hay algún error de usuario, como un formato de datos no válido.

In [24]:
try:
    response = client.transact_get_items(
        TransactItems=[
            {
                'Get': {
                    'TableName': 'Bookks',
                    'Key': {
                        'Author': {'S': 'William Shakespeare'},
                        'Title': {'S': 'The Tempest'}
                    }
                }
            },
            {
                'Get': {
                    'TableName': 'Bookks',
                    'Key': {
                        'Author': {'S': 'Franz Kafka'},
                        'Title': {'S': 'The Trial'}
                    }
                }
            }
        ]
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    print(json.dumps(response['Responses'], indent=4, default=str))

[
    {
        "Item": {
            "Title": {
                "S": "The Tempest"
            },
            "Formats": {
                "M": {
                    "Hardcover": {
                        "BOOL": true
                    },
                    "Paperback": {
                        "BOOL": true
                    },
                    "Audiobook": {
                        "BOOL": false
                    }
                }
            },
            "Author": {
                "S": "William Shakespeare"
            },
            "Category": {
                "S": "Drama"
            }
        }
    },
    {
        "Item": {
            "Title": {
                "S": "The Trial"
            },
            "Formats": {
                "M": {
                    "Ebook": {
                        "S": "ISBN 978-3-16-148408-7"
                    },
                    "Hardcover": {
                        "S": "ISBN 978-3-16-148410-0"
                    },
    